In [1]:
%pip install pyyaml

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import yaml # type: ignore
with open("../../../hosts", "r") as f:
    data = yaml.safe_load(f)

# 逐行打印加载的数据
# print(yaml.dump(data, allow_unicode=True))
# print(yaml.dump(data["ungrouped"]["hosts"]["default"]["ansible_host"], allow_u


192.227.212.51
...



In [ ]:
def parse_ssh_config(ssh_config_file):
    hosts = {}
    with open(ssh_config_file, 'r') as f:
        lines = f.readlines()
        current_host = None
        for line in lines:
            line = line.strip()
            if line.startswith('Host '):
                current_host = line.split()[1]
                hosts[current_host] = {}
            elif line.startswith('HostName'):
                hosts[current_host]['hostname'] = line.split()[1]
            elif line.startswith('User'):
                hosts[current_host]['user'] = line.split()[1]
            elif line.startswith('Port'):
                hosts[current_host]['port'] = line.split()[1]
    return hosts


def write_ansible_inventory(hosts, inventory_file):
    with open(inventory_file, 'w') as f:
        for host, params in hosts.items():
            f.write(f"[{host}]\n")
            f.write(f"{params['hostname']} ansible_user={params.get('user', 'your_user')} ansible_port={params.get('port', '22')}\n\n")


def write_ansible_cfg(remote_user, private_key_file, inventory_file, ansible_cfg_file):
    with open(ansible_cfg_file, 'w') as f:
        f.write("[defaults]\n")
        f.write(f"inventory = {inventory_file}\n")
        f.write(f"remote_user = {remote_user}\n")
        f.write(f"private_key_file = {private_key_file}\n")


def main():
    ssh_config_file = '/path/to/your/ssh/config'
    ansible_inventory_file = '/path/to/ansible/inventory'
    ansible_cfg_file = '/path/to/ansible.cfg'

    # SSH Config to Ansible Inventory
    hosts = parse_ssh_config(ssh_config_file)
    write_ansible_inventory(hosts, ansible_inventory_file)

    # Write ansible.cfg
    remote_user = 'your_ssh_user'
    private_key_file = '/path/to/your/private/key'
    write_ansible_cfg(remote_user, private_key_file, ansible_inventory_file, ansible_cfg_file)


if __name__ == "__main__":
    main()
